In [4]:
from Ionic_current.Ord11 import Ord11_model
from PermanentConstants import Constants
import torch
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float32

In [ ]:
const = Constants(
            device=device,
            dtype=dtype,
            Ncell=2,
            nbeats=1
            )
parameters = const.parameters

In [6]:
ti = 0.0
phi_i = const.phi0.clone()
G_i   = const.g0.clone()
S     = const.S.clone()
state = torch.cat((phi_i, G_i), dim=0)
N = parameters['N']

while ti < const.T:

    if abs(math.fmod(ti, const.parameters['bcl'])) < const.twin:
        dt = const.dt1; dt_samp = const.dt1_samp; Ns = const.Ns1
    else:
        dt = const.dt2; dt_samp = const.dt2_samp; Ns = const.Ns2

   
    if abs(math.fmod(ti, 500)) < 1e-9:
        print(f'time: {ti:.2f} ms')
    parameters['dt'] = dt

    Iion, _, G_new =  Ord11_model(ti, state, parameters, S, device, dtype)

    phi_new = phi_i + dt*(-Iion/parameters['Ctot'])

    state[0:N] = phi_new
    state[N:]  = G_new

    ti = round(ti + dt, 5) 

time: 0.00 ms
time: 500.00 ms


KeyboardInterrupt: 